In [1]:
%matplotlib inline 
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
from astropy.coordinates import SkyCoord
from sqlalchemy.dialects import postgresql

import hugs
import pandas as pd
from hugs.database.tables import Source, Tract, Patch

In [2]:
db_fn = '/tigress/jgreco/hugs-s18a/hsc-s18a-wide-20180920-133449/hsc-s18a-wide.db'

In [3]:
cat = Table.read('/tigress/jgreco/hugs-s18a/hsc-s18a-wide-20180920-133449/hugs-cat.csv')

In [33]:
from sqlalchemy import Integer

In [34]:
ids = tuple(cat['id'].astype(Integer))

In [5]:
engine = hugs.database.connect(db_fn)
session = hugs.database.Session()

In [192]:
query = session.query(Tract.hsc_id.label('tract'), Patch.hsc_id.label('patch'), 
                      Source.id, Source.ra, Source.dec, Source.flux_radius_i, 
                      Source.mag_auto_i).\
                      join(Patch, Patch.tract_id==Tract.id).\
                      join(Source, Source.patch_id==Patch.id).\
                      filter(Source.id.in_(ids))

#print(str(query.statement.compile(
#    dialect=postgresql.dialect(),
#compile_kwargs={"literal_binds": True})))

In [193]:
results = pd.read_sql(query.statement, engine)

In [197]:
tracts = []
for t in results.tract:
    tracts.append(np.frombuffer(t, np.int64)[0])

In [199]:
results['tract'] = tracts

In [200]:
len(np.unique(tracts))

489

In [201]:
len(results), len(cat)

(90931, 90931)

In [191]:
path = '/tigress/jgreco/hugs-s18a/hsc-s18a-wide-20180920-133449/'
results.to_csv(path + 'hugs-cat-with-tract-and-patch.csv', index=False)